In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Basic Recording Example
## Recording Terminology
- **Case**: A Case stores a snapshot of all the variable values, metadata, and options of a model, or a sub-set of a model, at a particular point in time
- **Case Recorder**: An OpenMDAO module used to store a snapshot of a model before, during, or after execution in an SQL file.
- **Sources**: The OpenMDAO object responsible for recording the case. Can be `Problem`, `Driver` or a specific `System` or `Solver` identified by pathname.

## Basic Recording Example

Below is a basic example of how to create a recorder, attach it to a Problem, save the information,
and retrieve the data from the recorder. `list_outputs` is a quick way to show all of your outputs
and their values at the time the case was recorded, and should you need to isolate a single value OpenMDAO provides two ways to
retrieve them. To view all the design variables, constraints, and
objectives, you can use their methods like the example below.


In [2]:
import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

In [3]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src8", get_code("openmdao.test_suite.components.sellar_feature.SellarMDAWithUnits"), display=False)

:::{Admonition} `SellarMDAWithUnits` class definition 
:class: dropdown

{glue:}`code_src8`
:::

In [4]:
from openmdao.test_suite.components.sellar_feature import SellarMDAWithUnits
import numpy as np
import openmdao.api as om

# build the model
prob = om.Problem(model=SellarMDAWithUnits())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]), upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

# setup the optimization
driver = prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

# Create a recorder variable
recorder = om.SqliteRecorder('cases.sql')
# Attach a recorder to the problem
prob.add_recorder(recorder)

prob.setup()
prob.set_solver_print(0)
prob.run_driver()
prob.record("after_run_driver")

# Instantiate your CaseReader
cr = om.CaseReader("cases.sql")
# Isolate "problem" as your source
driver_cases = cr.list_cases('problem', out_stream=None)
# Get the first case from the recorder
case = cr.get_case('after_run_driver')

# These options will give outputs as the model sees them
# Gets value but will not convert units
const = case['con1']
print(const)

# get_val can convert your result's units if desired
const_K = case.get_val("con1", units='K')

print(const_K)

[-1.66901604e-10]
[273.15]


In [5]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(const, -1.68550507e-10, 1e-3)
assert_near_equal(const_K, 273.15, 1e-3)

ValueError: actual [-1.66901604e-10], desired [-1.68550507e-10], rel error 0.009782844208595636, tolerance 0.001

In [ ]:
# list_outputs will list your model's outputs and return a list of them too
print(case.list_outputs())

In [ ]:
# This code below will find all the objectives, design variables, and constraints that the
# problem source contains
objectives = case.get_objectives()
print(objectives['obj'])

design_vars = case.get_design_vars()
print(design_vars['x'])

constraints = case.get_constraints()
print(constraints['con1'])

In [ ]:
assert_near_equal(objectives['obj'], 3.18339395, 1e-4)
assert_near_equal(design_vars['x'], 0., 1e-4)
assert_near_equal(constraints['con1'], -1.68550507e-10, 1e-4)